In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd
from datetime import datetime
import numpy as np
from requests_html import HTMLSession
import psycopg2 as pg2
from geopy import geocoders
from futures3 import ThreadPoolExecutor, as_completed
import swifter
import gensim
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import spacy
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from skillNer.skill_extractor_class import SkillExtractor
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
import plotly.graph_objs as go
from plotly.offline import plot
from sklearn.manifold import TSNE
import seaborn as sns
import pickle
import re
import os

In [ ]:
main_link = 'https://www.kariera.gr/en/jobs?title=&page=0&limit=20'
headers = {
    'User-agent':
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'
}
r = requests.get(main_link, headers=headers)
r

In [ ]:
host = os.environ.get("DB_HOST")
port = os.environ.get("DB_PORT")
database = os.environ.get("DB_NAME")
user = os.environ.get("DB_USER")
password = os.environ.get("DB_PASSWORD")

## Main Idea
### scrape all main_link + sub_links where sub_links=data%20analyst,data%20scientist,data%20engineer

In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
max_page = soup.find('li', class_='ant-pagination-next').previous_sibling.text
max_pages = int(max_page)

In [ ]:
max_pages

### working out how to turn pages:

In [ ]:
page_link_list = []
for i in range(0, max_pages + 1):
    page_link_list.append(
        'https://www.kariera.gr/en/jobs?title=&page={}&limit=20'.format(i))

## getting all sub-links from all links

In [ ]:
link_list = []
session = HTMLSession()

In [ ]:
def get_links(session, i):
    return [y for y in session.get(i).html.links if '/en/jobs/' in y]


link_list = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for i in tqdm(page_link_list):
        futures.append(executor.submit(get_links, session, i))
    for future in tqdm(as_completed(futures), total=len(futures)):
        link_list.extend(future.result())

In [ ]:
len(link_list)

In [ ]:
final_list = []
main = 'https://www.kariera.gr/'
for i in link_list:
    final_list.append(main + i)

# building the loop

In [ ]:
def process_page(link):
    unique_id = link[-5:]
    a = ''
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    if soup.find('div', class_='h4 UGhAL2CE') is not None:
        x = soup.find('div', class_='h4 UGhAL2CE').text
        if ('Data Analyst' in x) | ('Data Scientist' in x) | ('Data Engineer'
                                                              in x):
            try:
                job_title = soup.find('div', class_='h4 UGhAL2CE').text
            except:
                job_title = np.nan
            try:
                company = soup.find('a', class_='__8_RJM1pE').text
            except:
                company = np.nan
            try:
                location = soup.findAll(
                    'a', class_='V_oVSppD main-body-text')[0].text
            except:
                location = np.nan
            try:
                job_occupation = soup.findAll(
                    'a', class_='V_oVSppD main-body-text')[3].text
            except:
                job_occupation = np.nan
            try:
                level = soup.findAll('a',
                                     class_='V_oVSppD main-body-text')[1].text
            except:
                level = np.nan
            try:
                job_type = soup.findAll(
                    'a', class_='V_oVSppD main-body-text')[2].text
            except:
                job_type = np.nan
            try:
                result = soup.findAll('div', class_='__2DY4wJ3z hi8OBmAZ')
                for i in result:
                    a = i.get_text(strip=True, separator=' ')
                    a = a.replace('\u202f', '').replace('\xa0',
                                                        '').replace('/', '')
            except:
                content = np.nan
            temp = pd.DataFrame([{
                'job_id':
                unique_id,
                'job_title':
                job_title,
                'company':
                company,
                'location':
                location,
                'job_occupation':
                job_occupation,
                'level':
                level,
                'job_type':
                job_type,
                'content':
                a,
                'date_scraped':
                datetime.now().strftime("%Y-%m-%d %H:%M")
            }])
            return temp
        return None

In [ ]:
df2 = pd.DataFrame()
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for link in tqdm(final_list):
        futures.append(executor.submit(process_page, link))
    for future in tqdm(as_completed(futures), total=len(futures)):
        temp = future.result()
        if temp is not None:
            df2 = pd.concat([df2, temp], ignore_index=True)

## Connect to postgres

In [ ]:
port = int(port)
conn = pg2.connect(host=host,
                   port=port,
                   user=user,
                   password=password,
                   dbname=database)

In [ ]:
cur = conn.cursor()

In [ ]:
def create_table(cur):
    create_table_command = (""" 
    CREATE TABLE IF NOT EXISTS jobs (
    job_id int primary key NOT NULL,
   job_title text NOT NULL,
   company text,
   location text,
   job_occupation text,
   level text,
   job_type text,
   content text,
   date_scraped timestamp NOT NULL
    );
    """)

    cur.execute(create_table_command)

In [ ]:
create_table(cur)

In [ ]:
def check_if_job_exists(cur, job_id):
    query = (""" select job_id from jobs where job_id = %s """)
    cur.execute(query, (job_id, ))

    return cur.fetchone() is not None

In [ ]:
def update_db(cur, df):

    l = []
    for row in df.iterrows():
        if check_if_job_exists(cur, row[1][0]):
            pass
        else:
            l.append(row[1][0])

    return l

# Create dataframe with new jobs

In [ ]:
new_jobs = update_db(cur, df2)

In [ ]:
new_jobs = df2[df2['job_id'].apply(lambda x: x in new_jobs)]

In [ ]:
new_jobs

## Push new jobs to postgress

In [ ]:
def insert_into_table(cur, job_id, job_title, company, location,
                      job_occupation, level, job_type, content, date_scraped):
    insert_new_jobs = (
        """ INSERT INTO jobs (job_id, job_title, company, location, job_occupation, level,
           job_type, content, date_scraped)
            VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s);""")

    job_to_insert = (job_id, job_title, company, location, job_occupation,
                     level, job_type, content, date_scraped)

    cur.execute(insert_new_jobs, job_to_insert)

In [ ]:
def push_from_df_to_db(cur, df):
    for i, row in df.iterrows():
        insert_into_table(cur, row['job_id'], row['job_title'], row['company'],
                          row['location'], row['job_occupation'], row['level'],
                          row['job_type'], row['content'], row['date_scraped'])

In [ ]:
push_from_df_to_db(cur, new_jobs)

In [ ]:
conn.commit()

In [ ]:
conn.close()

# Fetch jobs table and clean duplicate entries

In [ ]:
conn = pg2.connect(host=host,
                   port=port,
                   user=user,
                   password=password,
                   dbname=database)

In [ ]:
with conn:

    cur = conn.cursor()
    cur.execute("SELECT * FROM jobs")

    rows = cur.fetchall()

df = pd.DataFrame(rows).rename(
    columns={
        0: 'job_id',
        1: 'job_title',
        2: 'company',
        3: 'location',
        4: 'job_occupation',
        5: 'level',
        6: 'job_type',
        7: 'content',
        8: 'date_scraped'
    })

In [ ]:
df.head()

## Dropping duplicates

In [ ]:
# function to clean and normalize the input data
def fingerprint(row):
    title = row['job_title'].lower().strip()
    company = row['company'].lower().strip()
    location = row['location'].lower().strip()
    return title + '-' + company + '-' + location


# create a new column 'fingerprint' and store the fingerprint values
df['fingerprint'] = df.apply(fingerprint, axis=1)

In [ ]:
df.drop_duplicates('fingerprint', inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop('fingerprint', axis=1, inplace=True)

In [ ]:
cur.execute("DELETE FROM jobs")

In [ ]:
push_from_df_to_db(cur, df)

In [ ]:
conn.commit()

# Fetch non duplicate entries

In [ ]:
cur.execute("SELECT * FROM jobs")

rows = cur.fetchall()

df = pd.DataFrame(rows).rename(
    columns={
        0: 'job_id',
        1: 'job_title',
        2: 'company',
        3: 'location',
        4: 'job_occupation',
        5: 'level',
        6: 'job_type',
        7: 'content',
        8: 'date_scraped'
    })

In [ ]:
conn.close()

## Cleaning company names

In [ ]:
df['company'] = df['company'].apply(lambda x: x.replace('\t', '').strip())

## Categorizing jobs

In [ ]:
df['categories'] = df['job_title'].apply(
    lambda x: 'Analytics'
    if 'Analyst' in x else ('Engineering' if 'Engineer' in x else 'Science'))

## Cleaning job_title

In [ ]:
# removing spaces
df['job_title'] = df['job_title'].apply(lambda x: x.strip())

In [ ]:
# removing anything inside paretheses (inclusive)
df['job_title'] = df['job_title'].apply(lambda x : re.sub(r'\s*\([^()]*\)', '', x))

In [ ]:
df['location'].replace('Athina','Athens',inplace=True)

In [ ]:
# removing locations from job titles
# it assumes that always all locations will be preceded by ' - '
def clean_city_from_job(job, city):
    titles = []
    for i, j in zip(job, city):
        if j in i:
            title = i[:-(3 + len(j))]
            titles.append(title)
        else:
            title = i
            titles.append(title)
    return pd.DataFrame(titles).rename(columns={0: 'job_title'})

In [ ]:
df = clean_city_from_job(df['job_title'], df['location']).merge(
    df, left_index=True,
    right_index=True).drop('job_title_y',
                           axis=1).rename(columns={'job_title_x': 'job_title'})

## Cleaning content

In [ ]:
df['content'] = df['content'].apply(
    lambda x: x.replace('Description', '').replace('Company', '').replace(
        'ΠΕΡΙΓΡΑΦΗ ΘΕΣΗΣ ΕΡΓΑΣΙΑΣ', '').replace('Job', '').replace(
            '’s', '').replace('Purpose', ''))

In [ ]:
df['content'] = df['content'].apply(
    lambda x: re.sub(r'Requisition ID: \d+', '', x))

## Geolocate locations

In [ ]:
gn = geocoders.GeoNames('asterios')

In [ ]:
df['geo'] = df['location'].swifter.apply(lambda x: gn.geocode(x)).apply(
    lambda loc: tuple(loc.point) if loc else None)

In [ ]:
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['geo'].tolist(),
                                                         index=df.index)

### At this point we are done with the cleaning next up is droping unwanted columns and exporting  the csv for the main app

In [ ]:
df.drop(['job_id', 'job_occupation', 'date_scraped', 'geo', 'altitude'],
        axis=1,
        inplace=True)

In [ ]:
df.to_csv('job_board.csv', index=False)

## Model Stuff

In [ ]:
# keep only job titles, company and content
df_c = df[['job_title', 'company', 'content']].copy()

# drop duplicates that evaded our previous fingerprint
df_c.drop_duplicates(['job_title', 'company'], keep='first', inplace=True)

## Clean content

# remove job titles from content
jobs = [i for i in df_c['job_title'].unique()] + ['Junior', 'Mid', 'Senior']

for i in jobs:
    df_c['content'] = df_c['content'].apply(lambda x: x.replace(i, ''))

# further cleaning
def clean_text(text):

    # Remove hashtags up to the leading whitespace
    text = re.sub(r'#\S+\s?', '', text)

    # Remove emails
    text = re.sub(r'\S+@\S+', '', text)

    # Remove links with https and www
    text = re.sub(r'(https?://\S+)|(www\.\S+)', '', text)

    # Remove special character ●
    text = re.sub(r'●', '', text)

    return text.strip()

df_c['content'] = df_c['content'].apply(lambda x: clean_text(x))

In [ ]:
## Skill extraction (NER) with skillNer database and spacy pretrained model

! python -m spacy download en_core_web_lg

# init params of skill extractor
nlp = spacy.load("en_core_web_lg")

def skill_extract(x):
    full = []
    ngrams = []

    try:
        # init skill extractor
        skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)
        # extract skills from job_description
        job_description = x
        annotations = skill_extractor.annotate(job_description)
        full = pd.DataFrame(
            annotations['results']['full_matches']).drop_duplicates(
                'skill_id')['doc_node_value'].tolist()
#         ngrams = pd.DataFrame(
#             annotations['results']['ngram_scored']).drop_duplicates(
#                 'skill_id').sort_values(
#                     'score', ascending=False)['doc_node_value'].tolist()
    except:
        np.nan

    return full

In [ ]:
# extract the skills
skills = []
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for i in df_c['content']:
        futures.append(executor.submit(skill_extract, i))
    for future in as_completed(futures):
        skills.extend([future.result()])

df_c['skills'] = skills

df_c['check'] = df_c['skills'].apply(lambda x: np.nan if len(x) == 0 else 0)

df_c.dropna(subset='check', inplace=True)

In [ ]:
# setting up for word2vec
sentences = df_c['job_title'].apply(lambda x: [x]) + df_c['skills']

model = Word2Vec(sentences, window=10, min_count=1, workers=4)

job_titles = [i for i in df_c['job_title'].unique()]

d = {}
for i in job_titles:
    try:
        d.update({str(i): model.wv[i]})
    except:
        d.update({str(i): np.nan})

final = pd.DataFrame(d).transpose()

In [ ]:
# clustering with KMeans
avg = {}
K = range(2, 20)

for num_clusters in K:
    kmeans = KMeans(n_clusters=num_clusters, n_init=50, random_state=42)
    kmeans.fit(final)
    score = silhouette_score(final, kmeans.labels_)
    avg[num_clusters] = score

max_score = max(avg.values())

In [ ]:
# cluster with the max avg silhouette score
best_cluster_num = max(avg, key=avg.get)

kmeans = KMeans(n_clusters=best_cluster_num)
kmeans.fit(final.values)

In [ ]:
# save clusters
final['cluster'] = kmeans.labels_

# join clusters with original df
skill_df = final['cluster'].reset_index().merge(
    df_c,
    right_on=df_c['job_title'],
    left_on=final['cluster'].reset_index()['index'])[[
        'job_title', 'skills', 'cluster'
    ]]

In [ ]:
# perform dimensionality reduction to 3D using t-SNE
tsne = TSNE(n_components=3, random_state=42, perplexity=final.shape[0] - 2)
embeddings_3d = tsne.fit_transform(final.values)

In [ ]:
# spit out necessary items for streamlit
df_c.to_csv('df_c.csv', index=False)
pickle.dump(avg, open('avg.pkl','wb'))
pickle.dump(kmeans, open('model.pkl','wb'))
pd.DataFrame(embeddings_3d).to_csv('embeddings_3d.csv', index=False)
skill_df.to_csv('skill_df.csv', index=False)